# View CellPainting data
This notebook shows how access ``Zarr`` files from an S3 bucket. 

## Load dependencies 

In [1]:
from IPython import get_ipython
from ipywidgets import *
import dask.array as da
import dask

import requests
import numpy as np
from datetime import datetime
from PIL import Image

from ome_zarr.io import parse_url
from ome_zarr.reader import Reader

## Location and data to view 

In [2]:
ENDPOINT_URL = "https://cellpainting-gallery.s3.amazonaws.com/"
ROOT = "cpg0004-lincs/broad/images/2016_04_01_a549_48hr_batch1/images_zarr/SQ00014812__2016-05-23T20_44_31-Measurement1.ome.zarr"

## Read information about the plate
Determine the number of rows and columns by reading the ``.zattrs`` file as JSON.¶ 

In [3]:
zattrs = requests.get(ENDPOINT_URL + ROOT + "/.zattrs").json()

### Helper methods 

In [4]:
def display_image(image, display_min, display_max): # copied from Bi Rico
    # https://stackoverflow.com/questions/14464449/using-numpy-to-efficiently-convert-16-bit-image-data-to-8-bit-for-display-with
    image.clip(display_min, display_max, out=image)
    image -= display_min
    np.floor_divide(image, (display_max - display_min + 1) / 256,
                    out=image, casting='unsafe')
    return image.astype(np.uint8)

def render_plane(dask_data, z, c, t, window=None):
    # -> 2D, also slice top/left quarter
    channel0 = dask_data[t, c, z, :1000, :1000]
    channel0 = channel0.compute()

    if window is None:
        window = [channel0.min(), channel0.max()]

    return display_image(channel0, window[0], window[1])


def setActiveChannels(dask_data, channels, colors, windows=None):
    # colors are (r, g, b)
    rgb_plane = None

    the_z = 0
    the_t = 0
    print("preparing image...")
    for ch_index, color in zip(channels, colors):
        plane = render_plane(dask_data, the_t, ch_index, the_z)
        if rgb_plane is None:
            rgb_plane = np.zeros((*plane.shape, 3), np.uint16)
        for index, fraction in enumerate(color):
            if fraction > 0:
                rgb_plane[:, :, index] += (fraction * plane)

    return rgb_plane.clip(0, 255, out=rgb_plane).astype(np.uint8)

# rgb = np.dstack((red, green, blue))
colors = [
    (0, 1, 1),
    (1, 1, 0),
    (1, 0, 1),
    (1, 0, 0),
    (0, 1, 0)
]

In [13]:
display_min = 0
display_max = 1000
def select_image(row=0, column=0, field=0):
    for well in zattrs['plate']['wells']:
        n = well['rowIndex']
        c = well['columnIndex']
        resolution=0
        if n == row and c == column:
            url = "%s%s/%s/%s/%s" % (ENDPOINT_URL, ROOT, well['path'], field, resolution)
            store = parse_url(url, mode="r").store
            reader = Reader(parse_url(url))
            # nodes may include images, labels etc
            nodes = list(reader())
            # first node will be the image pixel data
            dask_data = nodes[0].data[-1]
            rgb = setActiveChannels(dask_data, [c for c in range(dask_data.shape[1])], colors)
            img = Image.fromarray(rgb)
            display(img)
            

output = interactive(select_image, zattrs=zattrs,
                     row=widgets.IntSlider(value=0, min=0, max=len(zattrs['plate']['rows'])-1, step=1, description="Row", continuous_update=False),
                     column=widgets.IntSlider(value=0, min=0, max=len(zattrs['plate']['columns'])-1, step=1, description="Column", continuous_update=False),
                     field=widgets.IntSlider(value=0, min=0, max=zattrs['plate']['field_count']-1, step=1, description="Field", continuous_update=False))
display(output)

interactive(children=(IntSlider(value=0, continuous_update=False, description='Row', max=15), IntSlider(value=…

## License (BSD 2-Clause)

Copyright (C) 2022 University of Dundee. All Rights Reserved.

Redistribution and use in source and binary forms, with or without modification, are permitted provided that the following conditions are met:

Redistributions of source code must retain the above copyright notice, this list of conditions and the following disclaimer. Redistributions in binary form must reproduce the above copyright notice, this list of conditions and the following disclaimer in the documentation and/or other materials provided with the distribution. THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT OWNER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE. 